<a href="https://colab.research.google.com/github/BizStat/DecisionMaking/blob/main/CHAP11_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 4장 26쪽 재고문제

In [ ]:
from scipy.optimize import linprog
c = [400,400,400,400,450,450,450,450,20,20,20,20]
Aeq = [[1,0,0,0,1,0,0,0,-1,0,0,0],
       [0,1,0,0,0,1,0,0,1,-1,0,0],
       [0,0,1,0,0,0,1,0,0,1,-1,0],
       [0,0,0,1,0,0,0,1,0,0,1,-1]]
beq = [30,60,75,25]
bounds = [(0,40),(0,40),(0,40),(0,40),
          (0,None),(0,None),(0,None),(0,None),
          (0,None),(0,None),(0,None),(0,None)]
result = linprog(c, A_eq=Aeq, b_eq=beq,bounds=bounds)
print(result)

## 정수계획법

___
- pulp 라이브러리 설치

In [ ]:
!pip install pulp

In [ ]:
# prompt: A사는 은종(silver bell)과 은사슴(silver deer)를 생산,판매하고 있는데, 편의상 생산된 모든 상품은 팔린다고 가정. A사가 이용하는 두 가지 주요 자원은 은(silver)과 노동력으로, 회사의 이윤을 최대화하기 위하여 은종과 은사슴을 각각 몇 개씩 생산하는 것이 바람직한 것인지를 결정하려고 함. 은종 한 개를 팔면 10만원의 이익을, 은사슴은 6만원의 이익을 얻을 수 있음. 은종과 은사슴 한 개를 생산하기 위하여  각각 5kg, 2kg의 은과 각각 4시간의 노동력이 필요함. 다만 은종과 은사슴 생산에 사용할 수 있는 은은 20kg, 노동시간은 24시간으로 제한 됨. 생산량은 정수(integer)여야 함

from scipy.optimize import linprog

# 목적 함수: 이익 최대화 (은종 이익: 10만원, 은사슴 이익: 6만원)
c = [-10, -6]  # 최대화 문제이므로 계수에 -를 붙임

# 제약 조건: 은 사용량, 노동 시간 제한
A = [[5, 2], [4, 4]]
b = [20, 24]

# 변수의 범위 (생산량은 0 이상의 정수)
x_bounds = [(0, None), (0, None)]

# 선형 계획법 문제 해결
result = linprog(c, A_ub=A, b_ub=b, bounds=x_bounds, method='highs')

# 결과 출력
print(result)

# 정수 계획법을 위해 결과를 정수로 반올림
import numpy as np
optimal_x = np.round(result.x)
print("\nOptimal Solution (integer):")
print("Silver Bell:", int(optimal_x[0]))
print("Silver Deer:", int(optimal_x[1]))

# 정수 계획법을 사용한 결과 계산
profit = -np.dot(c, optimal_x)
print("Maximum Profit:", int(profit))

In [ ]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

# 문제 정의 (최대화 문제)
model = LpProblem(name="maximize_profit", sense=LpMaximize)

# 의사결정변수 정의 (정수 변수)
x1 = LpVariable(name="silver_bells", lowBound=0)
x2 = LpVariable(name="silver_deer", lowBound=0)

# 목적 함수 정의: 10x1 + 6x2
model += (10 * x1 + 6 * x2, "Profit")

# 제약조건 추가
model += 5 * x1 + 2 * x2 <= 20, "Silver constraint"
model += 4 * x1 + 4 * x2 <= 24, "Labor constraint"

# 최적화 실행
status = model.solve()

# 결과 출력
optimal_production = {var.name: var.value() for var in model.variables()}
maximum_profit = model.objective.value()

optimal_production, maximum_profit

In [ ]:
model

## 수송모형

In [ ]:
from pulp import *
TransModel = LpProblem(sense=LpMinimize)
x11 = LpVariable(name='서울->포항',lowBound=0, cat='Integer')
x12 = LpVariable(name='서울->대전',lowBound=0, cat='Integer')
x13 = LpVariable(name='서울->여수',lowBound=0, cat='Integer')
x21 = LpVariable(name='부산->포항',lowBound=0, cat='Integer')
x22 = LpVariable(name='부산->대전',lowBound=0, cat='Integer')
x23 = LpVariable(name='부산->여수',lowBound=0, cat='Integer')
x31 = LpVariable(name='광주->포항',lowBound=0, cat='Integer')
x32 = LpVariable(name='광주->대전',lowBound=0, cat='Integer')
x33 = LpVariable(name='광주->여수',lowBound=0, cat='Integer')
TransModel += 5*x11 + 4*x12 + 3*x13 + 8*x21 + 4*x22 + 3*x23 + 9*x31 + 7*x32 + 5*x33

TransModel += x11 + x12 + x13 == 100
TransModel += x21 + x22 + x23 == 300
TransModel += x31 + x32 + x33 == 300
TransModel += x11 + x21 + x31 == 300
TransModel += x12 + x22 + x32 == 200
TransModel += x13 + x23 + x33 == 200

status = TransModel.solve()

In [ ]:
LpStatus[status]

In [ ]:
TransModel

In [ ]:
x11.varValue

In [ ]:
value(TransModel.objective)

In [ ]:
optimal_trans = {var.name: var.value() for var in TransModel.variables()}
optimal_trans